In [ ]:
from pyspark.sql import SparkSession 

 
spark_session = SparkSession.builder\
        .master("spark://host-192-168-2-139:7077") \
        .appName("Linus app")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.memory", "2g")\
        .config("spark.cores.max", 8)\
        .getOrCreate()

In [ ]:

sc=spark_session.sparkContext
rdd=sc.textFile("./musicdata").map(lambda x: x.split(","))

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy
from pyspark.mllib.linalg import Vectors

In [ ]:
print("im hesre")
labeled_data_rdd = rdd.map(lambda x: [float(x[1]),[float(i.strip()) for i in x[2:-1]]+float[]])
print("im hesre")
labeled_data_rdd = labeled_data_rdd.map(lambda x: LabeledPoint(x[0], Vectors.dense(x[1])))
train_data_rdd, test_data_rdd = labeled_data_rdd.randomSplit([0.7, 0.3], seed=123)
print("im hesre")
nb_model = NaiveBayes.train(train_data_rdd)
print("im hesre")
predictions_rdd = nb_model.predict(test_data_rdd.map(lambda x: x.features))
print("im here")

In [ ]:
labels_and_predictions_rdd = test_data_rdd.map(lambda lp: lp.label).zip(predictions_rdd)
print("jjj")
metrics = MulticlassMetrics(labels_and_predictions_rdd)
print("sfsdfs")
accuracy = labels_and_predictions_rdd.filter(lambda lp: lp[0] == lp[1]).count() / float(labels_and_predictions_rdd.count())

print("Accuracy = {:.2f}%".format(accuracy * 100))